<h3> Training BiLSTM Model </h3>

In [19]:
import pandas as pd
import os
import re
import numpy as np
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import gensim
from gensim.models import Word2Vec

# BiLSTM Model Training

This notebook implements the BiLSTM model which will take TF-IDF based symptom representation and Word2Vec based symptom representation and predicts diagnoses codes.

---

- Input : symptom_disease_dict_{RUN_TAG}.json - Contains HADM_ID to Symptom text and Diagnosis mapping as json object
- Input : icd9_dict_{RUN_TAG}.json - Contains ICD9 Codes of TOP N Diagnoses
- Input : weight_i_j_norm{tag}.csv - TF-IDF weights for symptom representation

In [152]:
cwd = os.getcwd()
print(f"Current working directory : {cwd}")
# Let's define some constants that will be used below in our processing
MAX_NUMBER_OF_DISEASE = 100
MAX_SYMPTOMS = 50
BATCH_SIZE=400
WORD2VEC_NUM_FEATURES = 128
RUN_TAG = f"_{MAX_NUMBER_OF_DISEASE}_v2.0"
data_dir = cwd + "/../../data/"
SYMPTOM_DISEASE_DICT_FILE_PATH = data_dir + f"symptom_disease_dict_{RUN_TAG}.json"
ICD9_FILE_PATH = data_dir + f"icd9_dict_{RUN_TAG}.json"
SYMPTOM_DICT_FILE_PATH = data_dir + f"symptoms_dict_{RUN_TAG}.json"
TF_IDF_WEIGHTS_FILE_PATH = data_dir + f"weight_i_j_{RUN_TAG}.csv"
TF_IDF_NORM_WEIGHTS_FILE_PATH = data_dir + f"weight_i_j_norm_{RUN_TAG}.csv"
MODEL_DIR = data_dir + "/word2vec/"
# Word2Vec model will not change much - so keeping fixed file name
MODEL_FILE_PATH = MODEL_DIR + f"word2vec_model_sg_128__v2.0"
DISEASE_THRESHOLD = 0.20
SYMPTOM_TO_DIAGNOSE_MODEL_FILE= data_dir + f"symptom_diagnoses{RUN_TAG}.model"
SYMPTOM_TO_SYMPTOM_MODEL_FILE= data_dir + f"symptom_symptom{RUN_TAG}.model"

Current working directory : /Users/vijaymi/Studies/CS-598-DL4Health/Project/135-Disease-Inference-Method/disease_pred_using_bilstm/source


## 1. Loading all the required data

In [40]:
icd9_dict = None
with open(ICD9_FILE_PATH, 'r') as f:
            icd9_dict = json.load(f)
        
symptom_disease_dict = None
with open(SYMPTOM_DISEASE_DICT_FILE_PATH, 'r') as f:
            symptom_disease_dict = json.load(f)

symptoms_dict = None
with open(SYMPTOM_DICT_FILE_PATH, 'r') as f:
    symptoms_dict = json.load(f)

# Loading saved Word2Vec Model 
word2vec_model = gensim.models.Word2Vec.load(MODEL_FILE_PATH)
    
tfidf_weights = pd.read_csv(TF_IDF_NORM_WEIGHTS_FILE_PATH)
tfidf_weights.head()

,0,1,2,3,4,5,6,7,8,9,...,4016,4017,4018,4019,4020,4021,4022,4023,4024,4025
0,0.328974,0.0,0.0,0.348214,0.200701,0.0,0.0,0.0,0.0,0.817708,...,0.090909,0.666667,0.125,0.833333,0.666667,0.714286,0.500000,1.000,0.2,1.000
1,0.096579,0.0,0.0,0.171429,0.716152,0.0,0.0,0.0,0.0,0.802083,...,0.272727,0.000000,0.625,0.500000,0.111111,0.571429,0.333333,0.500,0.4,0.500
2,0.545272,0.0,0.0,0.498214,0.632686,0.0,0.0,0.0,0.0,1.000000,...,0.545455,0.666667,0.375,0.500000,0.666667,0.714286,0.833333,0.875,0.6,0.000
3,0.162978,0.0,0.0,0.300000,0.620580,0.0,0.0,0.0,0.0,0.739583,...,0.090909,0.166667,0.500,0.333333,0.222222,0.285714,0.166667,0.250,0.0,0.500
4,0.270624,0.0,0.0,0.294643,0.490284,0.0,0.0,0.0,0.0,0.630208,...,0.000000,0.166667,0.125,0.500000,0.111111,0.428571,0.500000,0.875,0.2,0.625


## 2. Symptom to Disease BiLSTM Model (for TF-IDF based Symptom representation)

### 2.1 Define Custom Loader for Symtom to Diagnoses based BiLSTM Model

In [141]:
from torch.utils.data import Dataset

class SymptomCustomDataset(Dataset):
    
    def __init__(self, filename, tfidf_weights, word2vec_model):        
        self.hadm_id_map = {}
        self.word2vec_embedding_cache = {}
        self.tfidf_embedding_cache = {}
        
        # TF-IDF Weights
        self.tfidf_weights = tfidf_weights
        
        # Word2Vec model
        self.model_w2v = word2vec_model.wv
        
        # Symptom dictionary
        with open(SYMPTOM_DICT_FILE_PATH, 'r') as f:
            self.symptom_dict = json.load(f)
        
        with open(ICD9_FILE_PATH, 'r') as f:
            self.icd9_dict = json.load(f)
        
        # read in the data files
        self.hadm_list = self.process_raw_data(filename)
        
        
    def process_raw_data(self, filename):
        symptom_disease_dict = None
        with open(filename, 'r') as f:
            symptom_disease_dict = json.load(f)
        hadm_list = []

        # Collecting all records for one admission as a tuple of symptoms list and diagnoses list
        # for hadm_id in list(symptom_disease_dict.keys())[:30000]:    
        for hadm_id in list(symptom_disease_dict.keys()):            
            symp_list, icd9_list = symptom_disease_dict[hadm_id]
            
            # TF-IDF - Matrix of size (MAX_SYMPTOMS, MAX_NUMBER_OF_DISEASE)
            tfidf_symp_vec = self.create_symptom_vector_with_tfidf(symp_list)
            if tfidf_symp_vec is None:
                continue
            
            # Word2Vec - Matrix of size (MAX_SYMPTOMS, MAX_NUMBER_OF_DISEASE)
            w2v_symp_vec = self.create_symptom_vector_with_word2vec(symp_list)
            
            # Processing Diagnoses Code - Series of size MAX_NUMBER_OF_DISEASE
            diag_vec = self.create_diagnosis_vector(icd9_list)

            # Save all three in hadm list
            hadm_list.append((tfidf_symp_vec, w2v_symp_vec, diag_vec, symp_list))
        
        return hadm_list
        
    def __len__(self):
        return len(self.hadm_list)
    
    def __getitem__(self, index):
        """
            Output:
            tf-idf symptpm_vector : number_of_diagnoses (50) x max_number_of_symptoms (50)
            word2vec symptpm_vector : max_number_of_symptoms (126) x number_of_diagnoses (50)
            diagnoses_vector = number_of_diagnoses
            symptom_count = number of symtoms for current record
        """
        tfidf_symp_vec, w2v_symp_vec, diag_vec, symp_list = self.hadm_list[index]
        
        # Create symptom vector for this admission record
        return torch.tensor(tfidf_symp_vec.T, dtype=torch.float), torch.tensor(w2v_symp_vec.T, dtype=torch.float), torch.tensor(diag_vec, dtype=torch.float), len(symp_list)
    
    def create_symptom_vector_with_tfidf(self, symp_list):
        # Convert Symptom text list to symptom index list
        symptom_idx_list = self.generate_tfidf_based_embedding(symp_list)
        
        if len(symptom_idx_list) == 0:
            return None
        
        symptom_vector = np.zeros((MAX_NUMBER_OF_DISEASE, MAX_SYMPTOMS))
        # Populate Symptom Vector by getting corresponding embeddings from TF-IDF vector
        for index, symptom_idx in enumerate(symptom_idx_list):
            # print(f"Symptom vector index: {index}, symptom index : {symptom_idx} \n {self.tfidf_weights.iloc[:,symptom_idx]}")
            symptom_vector[:, index] = self.tfidf_weights.iloc[:,symptom_idx]
        
        return symptom_vector
        

    def create_symptom_vector_with_word2vec(self, symp_list):
        # Create a 2d vector of symptoms (X)
        symp_vec = np.zeros((WORD2VEC_NUM_FEATURES, MAX_SYMPTOMS))
        
        # We take max of 50 symptoms from each discharge summaries note
        for index, symptom in enumerate(symp_list[:50]):
            symp_vec[:, index] = self.generate_word2vec_based_embedding(symptom)
        
        return symp_vec
        
    def generate_word2vec_based_embedding(self, symptom):
        
        # If present in cache, return from cache
        if symptom in self.word2vec_embedding_cache:
            return self.word2vec_embedding_cache[symptom]
        
        symp_words = symptom.split()
        symp_vec = np.zeros((WORD2VEC_NUM_FEATURES))
        for word in symp_words:
            # Get the word embedding
            key = re.sub(r"[^a-zA-Z0-9]","", word.lower())
            if key in self.model_w2v:
                symp_vec = symp_vec + self.model_w2v[key]
        symp_vec = symp_vec / len(symp_words)
        
        # Save embedding in cache for future
        self.word2vec_embedding_cache[symptom] = symp_vec
        
        return symp_vec
    
    # Converts symptom list to symptom vector
    def generate_tfidf_based_embedding(self, symp_list):            
        symp_index_list = []  
        # only consider notes with symptoms count more than 1
        if len(symp_list) > 1:
            for symptom in symp_list:
                if symptom in self.symptom_dict:
                    symp_index_list.append(self.symptom_dict[symptom])
        # print(f"symp_index_list[:MAX_SYMPTOMS] -- {symp_index_list[:MAX_SYMPTOMS]}")
        return symp_index_list[:MAX_SYMPTOMS]
    
    def create_diagnosis_vector(self, diagnoses_list):
        diag_vector = np.zeros((MAX_NUMBER_OF_DISEASE)) 
        # only consider notes with symptoms count more than 1
        for diagnoses in diagnoses_list:
            if diagnoses in self.icd9_dict:
                # self.icd9_dict[diagnoses] is the index of disease
                diag_vector[self.icd9_dict[diagnoses]] = 1

        return diag_vector   


In [142]:
dataset = SymptomCustomDataset(SYMPTOM_DISEASE_DICT_FILE_PATH, tfidf_weights, word2vec_model)
train_size = int(len(dataset)*0.8)
test_size = int(len(dataset)*0.1)
# train_size = 20000
# test_size = 2000
validation_size = len(dataset)  - (train_size + test_size)
train_dataset, test_dataset, validation_dataset = torch.utils.data.random_split(dataset, [train_size, test_size, validation_size])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [143]:
len(test_dataset)

5039

**Validate dataset created is with right size**

In [122]:
print(f"Dataset size : {len(dataset)}")
tfidf_symptom_rep, w2v_symptom_rep, diag_item, symptom_len = dataset[8]

print(f"tfidf_symptom_rep.shape : {tfidf_symptom_rep.shape}")
print(f"w2v_symptom_rep.shape : {w2v_symptom_rep.shape}")
print(f"diag_item.shape : {diag_item}")
assert tfidf_symptom_rep.shape == (MAX_SYMPTOMS, MAX_NUMBER_OF_DISEASE), "Incorrect Symptom representation shape."
assert w2v_symptom_rep.shape == (MAX_SYMPTOMS, WORD2VEC_NUM_FEATURES), "Incorrect Symptom representation shape."
assert diag_item.shape == (MAX_NUMBER_OF_DISEASE, ), "Incorrect diagnoses labels"
w2v_symptom_rep[0]

Dataset size : 30000
tfidf_symptom_rep.shape : torch.Size([50, 100])
w2v_symptom_rep.shape : torch.Size([50, 128])
diag_item.shape : tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=torch.float64)


tensor([ 4.6813e-02,  1.3691e-02, -2.1987e-03, -1.7290e-01,  3.2518e-01,
        -2.8748e-02,  2.7833e-02,  1.3619e-01,  1.2748e-01, -3.0961e-01,
        -2.4998e-01, -1.4167e-02,  1.6350e-01, -1.4159e-01,  6.9021e-02,
         1.9138e-01, -5.0848e-01,  3.0326e-01, -5.6162e-03,  3.5649e-01,
        -1.5401e-01, -6.5101e-02,  1.3688e-01,  6.0988e-02, -1.9457e-01,
        -4.5802e-02, -3.4792e-01,  1.4945e-01,  2.1323e-02,  1.3801e-01,
         1.1557e-01,  1.0230e-01,  3.5216e-01, -2.3127e-01,  4.3945e-01,
         2.8974e-01, -3.2591e-01,  2.0405e-01, -5.6875e-02, -2.0529e-02,
         3.4332e-01, -3.1652e-02, -2.4417e-01, -8.1900e-02,  1.2570e-02,
        -5.8611e-02, -2.6748e-01,  2.3834e-01,  1.7938e-01,  7.6852e-02,
         1.8585e-01,  1.3249e-01,  2.8239e-01, -4.1249e-01,  1.3322e-01,
        -2.3322e-01,  1.8220e-01, -1.5826e-01,  1.5149e-02, -2.2592e-02,
         1.0290e-01,  1.5922e-01, -7.5759e-02, -5.1029e-02,  1.5701e-01,
         4.9979e-01, -2.2250e-02,  2.7358e-01, -2.8

### 2.2 Define Symptom to Disease to BiLSTM Model (which uses TF-IDF based represenation)

In [79]:
class SymptomToDiseaseBiLstm(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, num_classes, batch_size):
        super(SymptomToDiseaseBiLstm, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.batch_size = batch_size
        
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=0.8)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, symp_length):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)

        # x = torch.nn.utils.rnn.pack_padded_sequence(x, symp_length, batch_first=True, enforce_sorted=False)
        
        out, (ht, ct) = self.bilstm(x, (h0, c0))
        
        # out, _ = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        out = self.fc(out[:,-1,:])
        out = self.sigmoid(out)
        
        return out

sym_disease_model = SymptomToDiseaseBiLstm(MAX_NUMBER_OF_DISEASE, 100, 2, MAX_NUMBER_OF_DISEASE, BATCH_SIZE)

### 2.3 Training and inferencing

In [53]:
def get_non_padded_pred_and_true_labels(y_pred, y_true, symptom_length_vector):
    
    print(f"y_pred.shape: {y_pred.shape}, y_true.shape: {y_true.shape}, symptom_length_vector: {symptom_length_vector.shape}")
    # Create a mask which will have all padded field to be zero
    
    mask_vector = np.ones(y_pred.shape)
    idx = 0
    for symptom_length in symptom_length_vector:
        mask_vector[idx,symptom_length:] = 0
        idx += 1
        
    mask = torch.tensor(mask_vector)
    mask_1 = mask.view(-1)
    mask_1 = mask_1.ge(1)
    
    y_pred_1 = y_pred.view(-1)
    y_true_1 = y_true.view(-1)
    
    y_pred_final = torch.masked_select(y_pred_1, mask_1)
    y_true_final = torch.masked_select(y_true_1, mask_1)
    
    print(f"y_pred_final.shape: {y_pred_final.shape}, y_true_final.shape: {y_true_final.shape}")
    return y_pred_final, y_true_final

In [93]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

def eval(model, test_loader, is_tf_idf_data):
    
    """    
    INPUT:
        model: model
        test_loader: dataloader
        
    OUTPUT:
        precision: overall micro precision score
        recall: overall micro recall score
        f1: overall micro f1 score
        
    REFERENCE: checkout https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
    """

    model.eval()
    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()
    for tf_idf_seq, word2vec_seq, labels, symp_len in test_loader:
        data = tf_idf_seq if is_tf_idf_data else word2vec_seq
        y_prob = model(data, symp_len)
        y_hat = (y_prob > DISEASE_THRESHOLD).int()      

        #y_hat, labels = get_non_padded_pred_and_true_labels(y_hat.detach().to('cpu'), labels.detach().to('cpu'), symp_len)
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, labels.detach().to('cpu')), dim=0)
        
    
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='micro')
    auc = roc_auc_score(y_true, y_pred, average='micro')
    return p, r, f, auc


In [106]:
def train(model, train_loader, test_loader, n_epochs, optimizer, crirerion, is_tf_idf_data):
    """    
    INPUT:
        model: the model
        train_loader: dataloder
        val_loader: dataloader
        n_epochs: total number of epochs
    """
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for tf_idf_seq, word2vec_seq, y_true, symp_len in train_loader:
            data = tf_idf_seq if is_tf_idf_data else word2vec_seq
                
            
            optimizer.zero_grad()
            y_hat = model(data, symp_len)
            
            loss = criterion(y_hat, y_true)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        p, r, f, auc = eval(model, test_loader, is_tf_idf_data)
        print('Epoch: {} \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, auc: {:.2f}'.format(epoch+1, p, r, f, auc))


In [118]:
import torch.optim as optim

sym_diag_optimizer = optim.Adam(sym_disease_model.parameters(), lr=0.001)
sym_diag_criterion = nn.BCELoss()

In [144]:
# number of epochs to train the model
n_epochs = 5

train(sym_disease_model, train_loader, test_loader, n_epochs, sym_diag_optimizer, sym_diag_criterion, True)

Epoch: 1 	 Training Loss: 0.239751
Epoch: 1 	 Validation p: 0.32, r:0.39, f: 0.35, auc: 0.66
Epoch: 2 	 Training Loss: 0.234632
Epoch: 2 	 Validation p: 0.32, r:0.40, f: 0.36, auc: 0.66
Epoch: 3 	 Training Loss: 0.230612
Epoch: 3 	 Validation p: 0.35, r:0.40, f: 0.37, auc: 0.67
Epoch: 4 	 Training Loss: 0.224973
Epoch: 4 	 Validation p: 0.36, r:0.44, f: 0.39, auc: 0.68
Epoch: 5 	 Training Loss: 0.219799
Epoch: 5 	 Validation p: 0.39, r:0.42, f: 0.40, auc: 0.68


#### Load Symptom to Disease Model

In [153]:
torch.save(sym_disease_model, SYMPTOM_TO_DIAGNOSE_MODEL_FILE)

<code>
Epoch: 1 	 Training Loss: 0.239751
Epoch: 1 	 Validation p: 0.32, r:0.39, f: 0.35, auc: 0.66
Epoch: 2 	 Training Loss: 0.234632
Epoch: 2 	 Validation p: 0.32, r:0.40, f: 0.36, auc: 0.66
Epoch: 3 	 Training Loss: 0.230612
Epoch: 3 	 Validation p: 0.35, r:0.40, f: 0.37, auc: 0.67
Epoch: 4 	 Training Loss: 0.224973
Epoch: 4 	 Validation p: 0.36, r:0.44, f: 0.39, auc: 0.68
Epoch: 5 	 Training Loss: 0.219799
Epoch: 5 	 Validation p: 0.39, r:0.42, f: 0.40, auc: 0.68
Epoch: 6 	 Training Loss: 0.239751
Epoch: 6 	 Validation p: 0.32, r:0.39, f: 0.35, auc: 0.66
Epoch: 7 	 Training Loss: 0.234632
Epoch: 7 	 Validation p: 0.32, r:0.40, f: 0.36, auc: 0.66
Epoch: 8 	 Training Loss: 0.230612
Epoch: 8 	 Validation p: 0.35, r:0.40, f: 0.37, auc: 0.67
Epoch: 9 	 Training Loss: 0.224973
Epoch: 9 	 Validation p: 0.36, r:0.44, f: 0.39, auc: 0.68
Epoch: 10 	 Training Loss: 0.219799
Epoch: 10 	 Validation p: 0.39, r:0.42, f: 0.40, auc: 0.68
</code>

## 3.0 Symptom to Symptom BiLSTM Model (for Word2Vec based symptom representation)

In [107]:
class SymptomToSymptomLstm(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(SymptomToSymptomLstm, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=0.8)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, symp_length):
        # Perform forward step
        out, (ht, ct) = self.bilstm(x)
        
        # print(f"out.shape : {out.shape}, ht.shape : {ht.shape}, ct.shape : {ct.shape}")
        # returning last layer of output from hidden state
        out = self.fc(out[:,-1,:])
        out = self.sigmoid(out)
        # Return model output
        return out

symp_to_symp_model = SymptomToSymptomLstm(128, 256, 2, MAX_NUMBER_OF_DISEASE)

In [130]:
import torch.optim as optim

sym_to_sym_optimizer = optim.Adam(symp_to_symp_model.parameters(), lr=0.001)
sym_to_sym_criterion = nn.BCELoss()

In [146]:

n_epochs = 5
train(symp_to_symp_model, train_loader, test_loader, n_epochs, sym_to_sym_optimizer, sym_to_sym_criterion, False)

Epoch: 1 	 Training Loss: 0.237278
Epoch: 1 	 Validation p: 0.34, r:0.40, f: 0.37, auc: 0.67
Epoch: 2 	 Training Loss: 0.232318
Epoch: 2 	 Validation p: 0.34, r:0.42, f: 0.37, auc: 0.67
Epoch: 3 	 Training Loss: 0.227837
Epoch: 3 	 Validation p: 0.36, r:0.42, f: 0.39, auc: 0.68
Epoch: 4 	 Training Loss: 0.221444
Epoch: 4 	 Validation p: 0.39, r:0.42, f: 0.40, auc: 0.68
Epoch: 5 	 Training Loss: 0.218801
Epoch: 5 	 Validation p: 0.37, r:0.45, f: 0.41, auc: 0.69


#### Save Symptom to symptom model on filesystem

In [150]:
torch.save(symp_to_symp_model, SYMPTOM_TO_SYMPTOM_MODEL_FILE)

<code>
Epoch: 1 	 Training Loss: 0.254184
Epoch: 1 	 Validation p: 0.28, r:0.32, f: 0.30, auc: 0.63
Epoch: 2 	 Training Loss: 0.250877
Epoch: 2 	 Validation p: 0.30, r:0.35, f: 0.32, auc: 0.64
Epoch: 3 	 Training Loss: 0.248819
Epoch: 3 	 Validation p: 0.30, r:0.35, f: 0.32, auc: 0.64
Epoch: 4 	 Training Loss: 0.247505
Epoch: 4 	 Validation p: 0.30, r:0.35, f: 0.33, auc: 0.64
Epoch: 5 	 Training Loss: 0.245524
Epoch: 5 	 Validation p: 0.30, r:0.35, f: 0.32, auc: 0.64
Epoch: 6 	 Training Loss: 0.237278
Epoch: 6 	 Validation p: 0.34, r:0.40, f: 0.37, auc: 0.67
Epoch: 7 	 Training Loss: 0.232318
Epoch: 7 	 Validation p: 0.34, r:0.42, f: 0.37, auc: 0.67
Epoch: 8 	 Training Loss: 0.227837
Epoch: 8 	 Validation p: 0.36, r:0.42, f: 0.39, auc: 0.68
Epoch: 9 	 Training Loss: 0.221444
Epoch: 9 	 Validation p: 0.39, r:0.42, f: 0.40, auc: 0.68
Epoch: 10 	 Training Loss: 0.218801
Epoch: 10	 Validation p: 0.37, r:0.45, f: 0.41, auc: 0.69
</code>

### 4.0 Combining two models with weighted mean

In [139]:
def performance_statistics(type_of_stat, y_true, y_pred, average='micro'):
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='micro')
    auc = roc_auc_score(y_true, y_pred, average='micro')
    print('{} \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, auc: {:.2f}'.format(type_of_stat, p, r, f, auc))
    return p, r, f, auc


def combined_prediction(sym_disease_model, symp_to_symp_model, test_loader, weight_of_tfidf):
    
    """    
    INPUT:
        model: model
        test_loader: dataloader
        
    OUTPUT:
        precision: overall micro precision score
        recall: overall micro recall score
        f1: overall micro f1 score
        
    REFERENCE: checkout https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
    """
    y_pred_1 = torch.LongTensor()
    y_pred_2 = torch.LongTensor()
    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()
    for tf_idf_seq, word2vec_seq, labels, symp_len in test_loader:

        y_prob_1 = sym_disease_model(tf_idf_seq, symp_len)
        y_prob_2 = symp_to_symp_model(word2vec_seq, symp_len)
        
        y_prob = y_prob_1 * weight_of_tfidf + y_prob_2 * (1.0 - weight_of_tfidf)
        
        y_hat_1 = (y_prob_1 > DISEASE_THRESHOLD).int()
        y_hat_2 = (y_prob_2 > DISEASE_THRESHOLD).int()
        y_hat = (y_prob > DISEASE_THRESHOLD).int()      

        #y_hat, labels = get_non_padded_pred_and_true_labels(y_hat.detach().to('cpu'), labels.detach().to('cpu'), symp_len)
        
        y_pred_1 = torch.cat((y_pred_1,  y_hat_1.detach().to('cpu')), dim=0)
        y_pred_2 = torch.cat((y_pred_2,  y_hat_2.detach().to('cpu')), dim=0)
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, labels.detach().to('cpu')), dim=0)
        
    performance_statistics("Symptom-Diag Statistics (TF-IDF)", y_true, y_pred_1)
    performance_statistics("Symptom-Sympton Statistics (Word2Vec)", y_true, y_pred_2)
    p, r, f, auc = performance_statistics("Combined (Word2Vec)", y_true, y_pred)

    return p, r, f, auc

In [147]:
p, r, f, auc = combined_prediction(sym_disease_model, symp_to_symp_model, test_loader, 0.5)

print('\nFinal \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, auc: {:.2f}'.format(p, r, f, auc))

Symptom-Diag Statistics (TF-IDF) 	 Validation p: 0.39, r:0.42, f: 0.40, auc: 0.68
Symptom-Sympton Statistics (Word2Vec) 	 Validation p: 0.37, r:0.45, f: 0.41, auc: 0.69
Combined (Word2Vec) 	 Validation p: 0.39, r:0.44, f: 0.41, auc: 0.69

Final 	 Validation p: 0.39, r:0.44, f: 0.41, auc: 0.69


<code>
Symptom-Diag Statistics (TF-IDF) 	 Validation p: 0.39, r:0.42, f: 0.40, auc: 0.68
Symptom-Sympton Statistics (Word2Vec) 	 Validation p: 0.30, r:0.35, f: 0.32, auc: 0.64
Combined (Word2Vec) 	 Validation p: 0.37, r:0.41, f: 0.39, auc: 0.68

Final 	 Validation p: 0.37, r:0.41, f: 0.39, auc: 0.68
</code>